# Fraud Detection Model for Supply Chain Finance

This notebook implements a machine learning model for detecting potential fraud in supply chain finance transactions.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

## Load and Preprocess Data

In [ ]:
# Load transaction data
df = pd.read_csv('../data/finance_transactions.csv')

# Feature engineering
df['transaction_hour'] = pd.to_datetime(df['timestamp']).dt.hour
df['transaction_day'] = pd.to_datetime(df['timestamp']).dt.dayofweek
df['amount_log'] = np.log1p(df['amount'])

# Prepare features
features = ['amount_log', 'transaction_hour', 'transaction_day', 'supplier_score',
           'buyer_score', 'historical_transaction_count']
X = df[features]
y = df['is_fraud']

## Train Model

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost model
model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train_scaled, y_train)

## Evaluate Model

In [ ]:
# Make predictions
y_pred = model.predict(X_test_scaled)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Feature Importance

In [ ]:
# Plot feature importance
importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
})
importance = importance.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importance)
plt.title('Feature Importance')
plt.show()